In [1]:
import requests
import json
import pandas as pd
from fake_useragent import UserAgent
from time import sleep
from tqdm import tqdm
import re

In [2]:
#'https://api.hh.ru/areas' # ссылка ID городов
#ID Москвы - 1

In [3]:
base_url = 'https://api.hh.ru/vacancies' #ссылка на API HH
agent = UserAgent().chrome # имитируем работы браузера
vac_list = [] #Будущий список вакансий
vac_name = 'Data Science' #Название вакансии, которое мы ищем

In [4]:
for page in tqdm(range(8)): # Собираем вакансии по страницам HH плюс прикручиваем полоску загрузки через tqdm
    params = {
        'text' : vac_name,
        'area' : 1,
        'per_page': 50,
        'page': page
    } # параметры для апи, где text - Текст вакансии, area - индекс города, который нам нужен из https://api.hh.ru/areas 
    #(1 это Москва), а дальше количество на страницу и номер страницы
    req = requests.get(base_url, headers = {'User-Agent' : agent}, params = params)
    j_req = req.json()# получаем питоновский словарь со списком вакансий по страницам
    vac_list.append(j_req) 
    sleep(0.5)# чтобы не ддосить сайт даем ему немного отдохнуть
    

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.16it/s]


In [5]:
id_list = [] #делаем список ID вакансий
for items in vac_list: #обход словарей с вакансиями на странице
    for vacs in items['items']: # обход списка словарей вакансий
        id_list.append(vacs['id'])

In [6]:
# Создаем будущую таблицу с интересующими нас данными
table = {'ID' : [], 'Наименование вакансии' : [], 'Зарплата' : [],'Валюта' : [], 'Опыт' : [], 
         'Адрес' : [],'Форма занятости' : [], 'Тип занятости' : [], 'Ключевые навыки' : [], 
         'Описание вакансии' : [], 'Специализация компании' : [], 'Дата публикации' : [], 'Ссылка на hh.ru': []}

In [7]:
for id in tqdm(id_list):
    info = requests.get(f'https://api.hh.ru/vacancies/{id}', headers = {'User-Agent' : agent})# для доступа к информации вакансии через ID
    j_info = info.json()
    table['ID'].append(j_info['id'])
    table['Наименование вакансии'].append(j_info['name'])
    if j_info['salary'] == None: #Форматирование зарплаты
        table['Зарплата'].append('Не указано')
    else:
        pay = j_info['salary']
        f_pay = f"От {pay['from']}"
        if pay['to'] != None:
            f_pay = f_pay + f" до {pay['to']}"
        table['Зарплата'].append(f_pay)
    if j_info['salary'] == None: #Форматирование стлобца валюты
        table['Валюта'].append('Не указано')
    else:
        table['Валюта'].append(j_info['salary']['currency'])
    if j_info['experience'] == None:#Форматирование Опыта
        table['Опыт'].append('Не указано')
    else:
        table['Опыт'].append(j_info['experience']['name'])
    if j_info['address'] == None: #Форматирование адреса (у HH жестокая инфа по адресам...)
        table['Адрес'].append('Не указано')
    else:
        try:
            address = f"{j_info['address']['city']}, {j_info['address']['street']}, {j_info['address']['building']}. Метро {j_info['address']['metro']['station_name']}."
        except:
            try:
                address = f"{j_info['address']['city']}, {j_info['address']['street']}, \
    {j_info['address']['building']}."
            except:
                try:
                    address = f"{j_info['address']['city']}, {j_info['address']['street']}."
                except:
                    try:
                        address = f"{j_info['address']['city']}."
                    except:
                        address = None
        table['Адрес'].append(address)
    if j_info['schedule'] == None: #Форматирование формы занятости
        table['Форма занятости'].append('Не указано')
    else:
        table['Форма занятости'].append(j_info['schedule']['name'])
    if j_info['employment'] == None: #Форматирование типа занятости
        table['Тип занятости'].append('Не указано')
    else:
        table['Тип занятости'].append(j_info['employment']['name'])
    if len(j_info['key_skills']) == 0:# Форматируем ключевые навыки
        table['Ключевые навыки'].append('Не указано')
    else:
        skill_list = []
        for skill in j_info['key_skills']:
            skill_list.append(skill['name'])
            skill_str = (', '.join(skill_list))
            skill_str = skill_str[:-1]+'.'
        table['Ключевые навыки'].append(', '.join(skill_list))
    table['Описание вакансии'].append(re.sub('<[^>]*>', '', j_info['description'])) #Форматируем удалением всех тэгов через регулярные строки
    spec_list = [] # Форматирование направлений компании
    for spec in j_info['specializations']:
        spec_list.append(spec['name'])
    table['Специализация компании'].append(', '.join(spec_list))
    time = j_info['published_at'].split('T') #Форматирование даты в ДД.ММ.ГГГГ '2022-07-03T13:35:19+0300'
    time_list = time[0].split('-')
    time = f"{time_list[2]}.{time_list[1]}.{time_list[0]}"
    table['Дата публикации'].append(time) 
    table['Ссылка на hh.ru'].append(j_info['alternate_url'])
    sleep(0.5)

100%|████████████████████████████████████████████████████████████████████████████████| 377/377 [03:55<00:00,  1.60it/s]


In [8]:
df = pd.DataFrame(table) #Выведем данные в табличный вид через pandas
df.head()

,ID,Наименование вакансии,Зарплата,Валюта,Опыт,Адрес,Форма занятости,Тип занятости,Ключевые навыки,Описание вакансии,Специализация компании,Дата публикации,Ссылка на hh.ru
0,66933586,Data scientist,От 200000 до 250000,RUR,От 3 до 6 лет,Не указано,Удаленная работа,Полная занятость,"Python, SQL, Pandas, Математическая статистика...","AgentApp — платформа для бизнеса, которая позв...","Другое, Аналитик, Аналитик, Другое",02.07.2022,https://hh.ru/vacancy/66933586
1,66875865,Data scientist в высокотехнологичный продукт,От 200000 до 400000,RUR,От 3 до 6 лет,Не указано,Полный день,Полная занятость,Не указано,Привет! Тебе интересна наука/технологии? Х...,"Программирование, Разработка, Системы управлен...",03.07.2022,https://hh.ru/vacancy/66875865
2,66692868,Head of Data (Dubai),От 14000,USD,Более 6 лет,Не указано,Полный день,Полная занятость,"Data Analysis, Management Reporting, Business ...",About us Mondia Digital empowers global busine...,"Информационные технологии, Управление проектами",02.07.2022,https://hh.ru/vacancy/66692868
3,67132810,Middle Data Scientist,От 200000 до 250000,RUR,От 3 до 6 лет,Не указано,Удаленная работа,Полная занятость,"Python, SQL, Hadoop, Machine Learning, Английс...","Привет, будущий коллега! Мы - команда социал...","Другое, Аналитик, Аналитик, Другое",02.07.2022,https://hh.ru/vacancy/67132810
4,67376016,Python Developer (Data Science),Не указано,Не указано,От 1 года до 3 лет,Не указано,Удаленная работа,Полная занятость,"Python, PostgreSQL, Git","ГК ИНТЕК - IT-компания, ведущий контент и серв...","Программирование, Разработка, Системы управлен...",03.07.2022,https://hh.ru/vacancy/67376016


In [9]:
df.to_csv('HH_parsing.csv', encoding = 'UTF-8', sep = ';') #и сохраняем в табличный файл с кодировкой и разделителями